In [1]:
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import fbeta_score
from sklearn.externals import joblib

import numpy as np
import cPickle as pickle
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import sys

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 950M (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 5005)


In [2]:
paths = ['../models', '../rainforest', '../scripts']
for path in paths:
    if path not in sys.path:
        sys.path.append(path)
        
import dataset
from data import *
from data_generators_leo import get_data

cur = os.getcwd()
data_path = cur + '\\..\\data\\'

weather_labels = ['clear', 'cloudy', 'haze', 'partly_cloudy']
train_csv = get_class_data(train=True, label='weather')
val_csv = get_class_data(train=False, label='weather')

train_labels = np.asarray(train_csv)[:,1:5]
train_data = np.asarray(train_csv)[:,0]
all_zeros_train = [index for index, label in enumerate(train_labels) if sum(label) == 0]
if(all_zeros_train > 0):
    train_data = np.delete(train_data, all_zeros_train)
    train_labels = np.delete(train_labels, all_zeros_train, axis=0)

val_labels = np.asarray(val_csv)[:,1:5]
val_data = np.asarray(val_csv)[:,0]
all_zeros_val = [index for index, label in enumerate(val_labels) if sum(label) == 0]
if(all_zeros_val > 0):
    val_data = np.delete(val_data, all_zeros_val)
    val_labels = np.delete(val_labels, all_zeros_val, axis=0)

print(train_labels.shape)
print(train_data.shape)
print(val_labels.shape)
print(val_data.shape)

(28334L, 4L)
(28334L,)
(12144L, 4L)
(12144L,)


In [3]:
train_gen = get_data(train_data, '..\\data\\train-jpg', train_labels,
                                      flip=True, shift_x=5, shift_y=5, rot_range=10,
                                      elastic_trans=False, batch_size=8, balance_batches=True)

val_gen = get_data(val_data, '..\\data\\train-jpg', val_labels, batch_size=8)
 

In [4]:
# jpg_path = data_path + 'train-jpg\\*.jpg'
# filelist = glob.glob(jpg_path)

# train_data = np.array([np.array(Image.open(fname)) for fname in filelist if os.path.basename(fname) in train_names])
# print(train_data.shape)

In [5]:
# train_data = np.transpose(train_data, (0, 3, 1, 2))

#train_data = train_data.astype('float32')
#train_data = train_data / 255.0
#with open('train_jpg.pkl', 'wb') as outfile:
    #pickle.dump(train_data, outfile, pickle.HIGHEST_PROTOCOL)


In [6]:
# X_train = train_data[1:30479, :, :, :]
# y_train= train_labels[1:30479, :]
# X_val = train_data[30479::, :, :, :]
# y_val = train_labels[30479::, :]

In [7]:
model = Sequential()

model.add(BatchNormalization(input_shape=[3, 256, 256], axis=1))

model.add(Convolution2D(32, 3, 3, input_shape=[3, 256, 256], activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 3, 3, activation='relu'))
model.add(BatchNormalization(axis=1))
#model.add(Convolution2D(128, 3, 3, activation='relu'))
#model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(len(weather_labels), activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(),
             metrics=['accuracy', 'categorical_crossentropy'])

model.summary()

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=[3, 256, 2...)`
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 3, 256, 256)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 254, 254)      896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 254, 254)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 252, 252)      9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 252, 252)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 126, 126)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 124, 124)      18496     
__________

In [8]:
csv_logger = CSVLogger('run4_adam.csv')
lr_plateau = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.5)
checkpoint = ModelCheckpoint(filepath='E:/KI/Master/MLIP/Brainforest/WeatherTest/model.{epoch:02d}-{val_loss:.2f}.hdf5',
                             verbose=1, save_best_only=True)

model.fit_generator(train_gen, steps_per_epoch=len(train_data),
                    epochs=25, verbose=1,
                    callbacks=[csv_logger, lr_plateau, checkpoint],
                    validation_data=val_gen, validation_steps=len(val_data))

#model.fit(X_train, y_train, validation_data=(X_val, y_val), nb_epoch=25, batch_size=8, verbose=1, callbacks=[csv_logger, lr_plateau, checkpoint])

Epoch 1/25
   80/28334 [..............................] - ETA: 14853s - loss: 11.9724 - acc: 0.2484 - categorical_crossentropy: 11.9724

KeyboardInterrupt: 